# LLM Newspaper Comments Generation with LangChain and OpenAI - **Giskard**

In [ ]:
%pip install "giskard[llm]" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 65.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7

In [ ]:
%pip install "openai>=1" --upgrade

In [ ]:
%pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
%pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.2 MB/s eta 0:00:00


In [ ]:
import os

import openai
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAI

from giskard import Dataset, Model, scan, GiskardClient

In [ ]:
from google.colab import userdata
key = userdata.get('OpenAI')

sk-proj-6HykS6FdEOTzATgfZ5cWT3BlbkFJRYHLRcRX4lc3SU7IY27C


In [ ]:
# Set the OpenAI API Key environment variable.
OPENAI_API_KEY = userdata.get('OpenAI')
openai.api_key = userdata.get('OpenAI')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Display options.
pd.set_option("display.max_colwidth", None)

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv"

TEXT_COLUMN_NAME = "text"

PROMPT_TEMPLATE = """
'{text}' \n\n
As reader you have to critisize the authors of the article above starting now : I believe this article is really
"""

RANDOM_STATE = 11

In [ ]:
df = pd.read_csv(DATA_URL)
df_filtered = df[[TEXT_COLUMN_NAME]].sample(10, random_state=RANDOM_STATE, ignore_index=True)

In [ ]:
giskard_dataset = Dataset(df_filtered, target=None)

INFO:giskard.datasets.base:Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


In [ ]:
giskard_dataset = Dataset(df_filtered, target=None)

INFO:giskard.datasets.base:Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


In [ ]:
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=[TEXT_COLUMN_NAME],
)

llm = OpenAI(
    request_timeout=20,
    max_retries=100,
    temperature=0,
    model_name="gpt-3.5-turbo-instruct",
)

chain = LLMChain(prompt=prompt, llm=llm)

# Test the chain.
chain.invoke(df_filtered.loc[0, TEXT_COLUMN_NAME])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'text': 'lacking in detail and context. It fails to mention the specific rules and regulations regarding over-rates in cricket and how they are calculated. It also does not mention if this is a common occurrence for Sarfaraz Ahmed or if this is his first offense. Additionally, the article does not provide any quotes or statements from the Pakistan team or officials regarding the fine. It would have been beneficial to include their perspective on the situation. Furthermore, the article does not mention the impact of the fine on Sarfaraz Ahmed and the team, both financially and in terms of morale. Overall, this article is lacking in depth and does not provide a well-rounded understanding of the situation. '}

In [ ]:
giskard_model = Model(
    model=chain,
    model_type="text_generation",
    name="Comment generation",
    description="This model is a professional newspapers commentator.",
    feature_names=[TEXT_COLUMN_NAME]
)

INFO:giskard.models.automodel:Your 'model' is successfully wrapped by Giskard's 'LangchainModel' wrapper class.


In [ ]:
# Validate the wrapped model and dataset.
print(giskard_model.predict(giskard_dataset).prediction)

INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:giskard.utils.logging_utils:Predicted dataset with shape (10, 1) executed in 0:00:24.408831


['lacking in detail and context. It fails to mention the specific rules and regulations regarding over-rates in cricket and how they are calculated. Additionally, it does not provide any explanation for why Pakistan was three overs short of their target. Was there a specific reason for this, such as injuries or delays? Furthermore, the article does not mention if this is a common issue for the Pakistani team or if they have been fined for slow over-rates in the past. Without this information, it is difficult to fully understand the severity of the situation and the reasoning behind the fines. The article also does not mention the amount of the fines, which is important information for readers to know. Overall, this article lacks depth and leaves readers with more questions than answers. '
 "lacking in depth and substance. The author fails to provide any context or background information about Sushant Singh Rajput, leaving readers who are not familiar with him confused. Additionally, th

In [ ]:
results = scan(giskard_model, giskard_dataset)

🔎 Running scan…


INFO:giskard.scanner.logger:Running detectors: ['LLMBasicSycophancyDetector', 'LLMCharsInjectionDetector', 'LLMHarmfulContentDetector', 'LLMImplausibleOutputDetector', 'LLMInformationDisclosureDetector', 'LLMOutputFormattingDetector', 'LLMPromptInjectionDetector', 'LLMStereotypesDetector', 'LLMFaithfulnessDetector']


Estimated calls to your model: ~365
Estimated LLM calls for evaluation: 148

Running detector LLMBasicSycophancyDetector…


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https:

LLMBasicSycophancyDetector: 1 issue detected. (Took 0:01:18.320378)
Running detector LLMCharsInjectionDetector…


...

LLMInformationDisclosureDetector: 0 issue detected. (Took 0:01:17.428170)
Running detector LLMOutputFormattingDetector…


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}


LLMOutputFormattingDetector: 0 issue detected. (Took 0:00:00.369560)
Running detector LLMPromptInjectionDetector…


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:giskard.utils.logging_utils:Predicted dataset with shape (1, 1) executed in 0:00:01.502861
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:giskard.utils.logging_utils:Predicted dataset with shape (1, 1) executed in 0:00:01.247079
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:giskard.utils.logging_utils:Predicted dataset with shape (1, 1) executed in 0:00:02.305663
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HT

KeyboardInterrupt: 

In [ ]:
test_suite = results.generate_test_suite("Test suite generated by scan")
test_suite.run()